In [ ]:
import random
import time

def buildFactorBase(P,Q,orderP):
    factorBaseSize = ceil(orderP**(1/3))
    factorBase = []
    coord = []
    filled=0
    while filled < factorBaseSize:
        a = (int)(random.random()*orderP) #a in [0, orderE - 1]
        b = (int)(random.random()*orderP)
        cand=a*P+b*Q
        if cand not in factorBase:
            factorBase.append(cand)
            coord.append((a,b))
            filled += 1
    return coord, factorBase
#k.<a>=FiniteField(3^3, modulus=x^3-x-1)
#q.<b>=FiniteField(3^3, modulus=x^3-x+1)

def sumInF(coord, factorBase, orderP):
    length = len(factorBase)
    for i in range(0,length):
        for j in range(i+1,length):
            dp=[factorBase[i], factorBase[j]]
            for v in VectorSpace(GF(2),2):
                #test=[-factorBase[i] if v[0] else factorBase[i]]
                #test+=[-factorBase[j] if v[1] else factorBase[j]]
                test=sum((-1)^v[k]*dp[k] for k in range(0,2)) #simplify
                try:
                    id = factorBase.index(test)
#                     print("coord:",coord[id])
#                     print("v:",v)
#                     print("i,j", coord[i], coord[j])
#                     b = coord[id][1] 
#                     a = -coord[id][0] #a=(a_1 + a_2 - g_1)
#                     b += coord[i][1] if v[0] else -coord[i][1] + coord[j][1] if v[1] else -coord[j][1]
#                     a -= coord[i][0] if v[0] else -coord[i][0] + coord[j][0] if v[1] else -coord[j][0] 
                    b = coord[id][1] 
                    a = -coord[id][0] #a=(a_1 + a_2 - g_1)
                    if v[0]:
                        b+=coord[i][1]
                        a-=coord[i][0]
                    else:
                        b-=coord[i][1]
                        a+=coord[i][0]
                    
                    if v[1]:
                        b+=coord[j][1]
                        a-=coord[j][0]
                    else:
                        b-=coord[j][1]
                        a+=coord[j][0]
                    try:
                        res=mod(a*inverse_mod(b, orderP), orderP)
#                        print(a,b)
                        return res
                    except ZeroDivisionError: #inverse of 'b' does not exist
                        pass
                except ValueError:
                    pass
    return -1
                
#prime
#11 | 13 | 17 | 19 | 23 | 29 | 31 | 37 | 41 | 43 | 47 | 53 | 59 | 61 | 67 | 71 | 73 | 79 | 83 | 89 | 97
p=23
T.<a> = GF(2**p)
while True:
    coefs = [] 
    for i in range(0,5):
        coefs.append( (int)(random.random() >= 0.5) )
    try:
        E = EllipticCurve(T, coefs)
        break
    except ArithmeticError: #if E singular
        pass
print(E)
P=E.gen(0) #base point P
orderP=P.order()
print("Order of P is {}.".format(orderP))

#Q = kP
k=(int)(1+(orderP-1)*random.random()) #k in [1, orderP-1]
Q=k*P
# print(P)
# print(Q)
# print(coord)

# print(factorBase)
attempts=0

start = time.time()
r1=discrete_log(Q,P, E.order(), operation='+')
print("It took {:4.4f} seconds. True answer: {}, Sage_algo_answer: {}."\
      .format((time.time()-start), k, r1))
print("Factorization of Elliptic curve order: {} = {}".format(orderP, orderP.factor()))
start = time.time()
while True:
    coord, factorBase = buildFactorBase(P,Q,orderP)
    attempts += 1
#    if mod(attempts,20) == 1:
    print("Attempt {}, Factor base of size {} builded.".format(attempts, len(factorBase)) )
    res=sumInF(coord, factorBase, orderP)
    if res != -1 or attempts > 1e3:
        break
print("It took {} attempts and {:4.4f} seconds. True answer: {}, algo_answer: {}."\
      .format(attempts,(time.time()-start), k, res))

In [103]:
#Individual step of Pohlig-Hellman algorithm
#Solves DLP in a subgroup of 'p^e' order
#https://courses.fit.cvut.cz/MI-MKY/media/lectures/mi-mky-poznamky-v17.pdf (page 81 PDF)
#Pi - generator of the subgroup, Qi = ki*Pi, order of Pi is p^e, returns ki
#Splits ki = x0 + p*x1 + p^2*x2 + p^(e-1)*x_(e-1) and solves 'e' DLP in a subgroup of order 'p'
def PH_reduction(Qi, Pi, p, e): 
    invPi = -Pi #additive inverse
    Pi = (p^(e-1))*Pi
    ki = 0 #ki = x0 + p*x1 + p^2*x2 + ...
    for i in range(0, e):
        tempQ = (p^(e-1-i))*(Qi + ki*invPi)
        ki += (p^i)*discrete_log_rho(tempQ, Pi, operation="+")
    return ki


#Implements Pohlig-Hellman algorithm
#https://courses.fit.cvut.cz/MI-MKY/media/lectures/mi-mky-poznamky-v17.pdf (page 81 PDF)
#Solves Q = k*P, returns k
#P generates a subgroup of order orderP
def Pohlig_Hellman_additive(Q, P, orderP):
    print("Factorization of orderP: {} = {}.".format(orderP,orderP.factor()))
    ki = [] #list of individual DLP results
    mi = [] #list of moduli
    for p in prime_factors(orderP): #prime factor p
        e = orderP.valuation(p) #multiplicity of p in orderP
        mi.append(p^e) 
        pei = int(orderP/mi[-1])
        ki.append(PH_reduction(pei*Q, pei*P, p, e)) #solves DLP in subgroup <(p^e)*P>
    k = CRT(ki, mi)

    assert(k*P == Q) #reality check
    return k


In [104]:
import random
import time

#implement pohling hellman and use rho
p=47
T.<a> = GF(2**p)
while True:
    coefs = []
    for i in range(0,5):
        coefs.append( (int)(random.random() >= 0.5) )
    try:
        E = EllipticCurve(T, coefs)
        break
    except ArithmeticError: #if E singular
        pass
print(E)
P=E.gen(0) #base point P
orderP=P.order()
print("Order of P is {}. Bit security (log2 of the biggest prime_factor of orderP): {:4.1f}"\
      .format(orderP, float(log(max(prime_factors(orderP)))/log2)))

k=(int)(1+(orderP-1)*random.random()) #k in [1, orderP-1]
Q=k*P

start = time.time()
res = Pohlig_Hellman_additive(Q,P,orderP)
print("To solve ECDLP it {} took {:4.4f} seconds, result isValid: {}. Method PH and Polard-rho."\
      .format(orderP, (time.time()-start), bool(res == k)))

start = time.time()
res = discrete_log(Q,P,orderP, operation="+")
print("To solve ECDLP it {} took {:4.4f} seconds, result isValid: {}. Method PH and BsGs - Sage default."\
      .format(orderP, (time.time()-start), bool(res == k)))


Elliptic Curve defined by y^2 + y = x^3 + 1 over Finite Field in a of size 2^47
Order of P is 140737488355329. Bit security: 37.3
Factorization of orderP: 140737488355329 = 3 * 283 * 165768537521.
To solve ECDLP it 140737488355329 took 71.9261 seconds, result isValid: True. Method PH and Polard-rho.
To solve ECDLP it 140737488355329 took 66.0104 seconds, result isValid: True. Method PH and BsGs - Sage default.


In [ ]:
#https://www.maths.unsw.edu.au/sites/default/files/mandyseetthesis.pdf (page 76)
########################################################################
# Author: Dr. D. Kohel, M. Seet #
# #
# SAGE Mathematical Software, Version 2.6, http://www.sagemath.org #
# #
########################################################################